# Find minimum f_c required to pass

In [1]:
import os
import pandas as pd
import numpy as np
import _canoeValues as cv

"""

Ask Trevor how to incorporate reinforcement strength in punching shear calculation

"""

#-------------------Inputs-------------------------------------------------------------
paddler_mass = cv.pad_male # Max paddler mass [kg]
g = cv.g # Gravity [m/s^2]
paddler_weight = paddler_mass*g # Max paddler weight [N]
safety_factor = cv.safety_factor
fac_weight = safety_factor*paddler_weight # Factored Weight [N]
A = cv.kneecap_length # Punching area length [mm]
B = cv.kneecap_width # Punching area width [mm]
layer1 = cv.hasLayer1 # if layer exists, it equals to 1. Otherwise, zero.
layer2 = cv.hasLayer2 # if layer exists, it equals to 1. Otherwise, zero.
d_mesh_1 = cv.d_mesh_1 # distance b/w the first layer of reinforcement to the compression surface [mm] (distance from where force is applied)
d_mesh_2 = cv.d_mesh_2 # distance b/w the second layer of reinforcement to the compression surface [mm] (distance from where force is applied)
# f_c = cv.compressive_strength # Compressive strength of concrete [MPa] (TODO: try tensile)
lamda = cv.lamda # Factor to account for lower tensile strength of low-density concrete
phi_c = cv.phi_c # Safety factor for concrete
alpha_s = cv.alpha_s # As per CSA A23.3-14 CL 13.3.4.1 (13-7)
#--------------------------------------------------------------------------------------

print(cv.d_mesh_1)

for k in range(0, 10000):

    f_c = k/10

    # Set d to the midpoint between all the layers of reinforcement if they exist
    # (assumes all layers of reinforcement are in the tension zone)
    # NOTE: put punching shear reinforcement in tension zone or failure will occur
    if layer2 == 0:
        d = d_mesh_1
    else:
        d = (d_mesh_1+d_mesh_2)/2

    # Calculate the area
    # Perimeter of punching area on tension side
    # Considered a rect: Side 1: (A + d) Side 2: (B + d)
    b0 = 2*A + 2*B + 4*d # [mm]
    area = d*b0 # [mm^2]

    # Ratio of long side (A) to short side (B) of a column section
    beta_c = A/B
    if beta_c<1:
        print('Warning: Beta_c is smaller than one.')

    # Shear stresses resisted
    Vc_1 = (1+2/beta_c)*0.2*lamda*phi_c*np.sqrt(f_c)    
    Vc_2 = ((alpha_s*d/b0)+0.19)*lamda*phi_c*np.sqrt(f_c)
    Vc_3 = 0.38*lamda*phi_c*np.sqrt(f_c)
    Vc = min(Vc_1, Vc_2, Vc_3)

    # Shear stresses applied
    Vf = fac_weight/area

    if Vc>=Vf:
        print('The design is good for punching shear.')
        print('The minimum value of f_c required is ' + str(f_c) + " MPa")
        break
    else:
        continue
        # print('The design needs to be re-done for punching shear.')

print("\n\n\n", str(Vc_1))
print(str(Vc_2))
print(str(Vc_3))
print(str(b0))
print(fac_weight)

8
The design is good for punching shear.
The minimum value of f_c required is 10.5 MPa



 0.9478083271421497
0.5180262753518301
0.6002786071900281
232
961.06608


# Determine if current mix passes

In [2]:
# INPUT
f_c = cv.compressive_strength
# -------------------------------------------------------------------------------------------

# Set d to the midpoint between all the layers of reinforcement if they exist
# (assumes all layers of reinforcement are in the tension zone)
# NOTE: put punching shear reinforcement in tension zone or failure will occur
if layer2 == 0:
    d = d_mesh_1
else:
    d = (d_mesh_1+d_mesh_2)/2

# Calculate the area
# Perimeter of punching area on tension side
# Considered a rect: Side 1: (A + d) Side 2: (B + d)
b0 = 2*A + 2*B + 4*d # [mm]
area = d*b0 # [mm^2]

# Ratio of long side (A) to short side (B) of a column section
beta_c = A/B
if beta_c<1:
    print('Warning: Beta_c is smaller than one.')

# Shear stresses resisted
Vc_1 = (1+2/beta_c)*0.2*lamda*phi_c*np.sqrt(f_c)    
Vc_2 = ((alpha_s*d/b0)+0.19)*lamda*phi_c*np.sqrt(f_c)
Vc_3 = 0.38*lamda*phi_c*np.sqrt(f_c)
Vc = min(Vc_1, Vc_2, Vc_3)

# Shear stresses applied
Vf = fac_weight/area

print("Vc: " + str(Vc))
print("Vf: " + str(Vf))


if Vc>=Vf:
    print('The design is good for punching shear.')
else:
    print('The design needs to be re-done for punching shear.')
print(str(area))

Vc: 0.5302167967650493
Vf: 0.517815775862069
The design is good for punching shear.
1856


# Honestly unsure why this last part exists anymore

In [3]:
# INPUT
f_c = cv.compressive_strength
fac_weight = safety_factor*(paddler_weight*2) # Factored Weight [N]
# -------------------------------------------------------------------------------------------

# Set d to the midpoint between all the layers of reinforcement if they exist
# (assumes all layers of reinforcement are in the tension zone)
# NOTE: put punching shear reinforcement in tension zone or failure will occur
if layer2 == 0:
    d = d_mesh_1
else:
    d = (d_mesh_1+d_mesh_2)/2

# Calculate the area
# Perimeter of punching area on tension side
# Considered a rect: Side 1: (A + d) Side 2: (B + d)
b0 = 2*A + 2*B + 4*d # [mm]
area = d*b0 # [mm^2]

# Ratio of long side (A) to short side (B) of a column section
beta_c = A/B
if beta_c<1:
    print('Warning: Beta_c is smaller than one.')

# Shear stresses resisted
Vc_1 = (1+2/beta_c)*0.2*lamda*phi_c*np.sqrt(f_c)    
Vc_2 = ((alpha_s*d/b0)+0.19)*lamda*phi_c*np.sqrt(f_c)
Vc_3 = 0.38*lamda*phi_c*np.sqrt(f_c)
Vc = min(Vc_1, Vc_2, Vc_3)

# Shear stresses applied
Vf = fac_weight/area

print("Vc: " + str(Vc))
print("Vf: " + str(Vf))


if Vc>=Vf:
    print('The design is good for punching shear.')
else:
    print('The design needs to be re-done for punching shear.')


Vc: 0.5302167967650493
Vf: 1.035631551724138
The design needs to be re-done for punching shear.
